<a href="https://colab.research.google.com/github/JohnnyKyr/CompIntel-Projects/blob/main/GAs2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [275]:
from google.colab import drive
drive.mount('/drive')

import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from math import log2
import random
import numpy as np
import math
from scipy import sparse

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [276]:
class Parser:

    def __init__(self):
        self.dataMatrix = None
        self.labels = None
        self.size = None

    def getLexicalSize(self,file):
        lexiconS = []
        with open(file, "r") as fp:
            for line in fp:
                lexiconS.append(line.split()[0])
        return  len(lexiconS)
    def getLabels(self,data):
        self.labels = []
        with open(data, "r") as fp:
            for line in fp:
                self.labels.append([int(i) for i in line.split()])

    def getData(self,data,lexicon):
        bagof = []
        bagofcsr =[]
        with open(data,"r") as fp:
            for line in fp:
                vertex = [0 for i in range(self.getLexicalSize(lexicon))]
                array = list(filter(lambda x : x[0] != '<' , line.split()))
                for ele in array:
                    vertex[int(ele)] +=1
                self.size = len(vertex)
                bagof.append(vertex)
        return bagof


In [277]:

datalex = Parser()


bagofwords = datalex.getData("/drive/MyDrive/ML/train-data.dat","/drive/MyDrive/ML/vocabs.txt")



In [278]:

def tf_idf(bagofwords,verbose=0):
  count=0
  N=len(bagofwords)
  n = len(bagofwords[0])
  tfbag = []
  
  textDCount = np.count_nonzero(bagofwords,axis=0)  #word count of the text body
  worddcount = np.count_nonzero(bagofwords,axis=1) #word count for each text
  
  #----------------IDF----------------------
  _idf = [0 for _ in range(len(textDCount))]
  tf_idf= []
  for k,value in enumerate(textDCount):
    _idf[k] = np.log10(N/value)
  _idf = np.array(_idf)
  #---------------TF-----------------------
  tfDict = [0 for _ in range(N)]
  for i in range(N):
    count+=1
    tfDict[i] = np.array([(text/worddcount[i]) for k,text in enumerate(bagofwords[i])])
    if verbose==1:
      print("======================================")
      print("Finished",count,"st","Calculation")
  
  tfDict = np.array(tfDict)
  
  
  for ele in tfDict:
    tf_idf.append(ele*_idf)
  
  avg = np.mean(tf_idf, axis = 0)
  return avg
   

tf__idf = tf_idf(bagofwords)

maximum = 0
for ele in tf__idf:
  maximum+=ele

In [316]:
class Individual(object):
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.fitness = 0

    @classmethod
    def create_genome(self):

        genome = np.random.randint(0, 1, size=8520)
        rand_ones = random.randint(0, 8519)
        for _ in range(rand_ones):
          index = random.randint(0, 8519)
          while genome[index] == 1:
            index = random.randint(0, 8519)
          else:
            genome[index] = 1
        return genome

    def calculate_fitness(self):
        return 0


def genome_rejection(genome: Individual):
    # replace the non valid genome
    # TODO recursion
    while np.count_nonzero(genome) < 1000:
        genome = Individual.create_genome()
    return genome


def repair_genome(genome: Individual):
    nnz = np.count_nonzero(genome)
    while nnz < 1000:
        difference = 1000 - nnz
        for _ in range(difference):
            genome[random.randint(0, 8519)] = 1
        nnz = np.count_nonzero(genome)
    return genome

def initialize_population(no_of_chromosomes):
    population = []
    for _ in range(no_of_chromosomes):
        genome = Individual.create_genome()
        # genome = genome_rejection(genome)
        genome = repair_genome(genome)
        population.append(Individual(genome))

    return population


def fitnessFunc(population:Individual,tfidf):
  #Calculating the fittest with the best sum of avg in tf_idf
  #Sort population by nonzero
  global maximum

  for i,individual in enumerate(population):
    for value in np.nonzero(individual.chromosome)[0]:
      individual.fitness += tfidf[value]

    if np.count_nonzero(individual.chromosome)>2000:
      per = (np.count_nonzero(individual.chromosome)//100)/100
      individual.fitness -=per*maximum
  
  

def RouleteWheelSelection(population):
  
  s=0

  population_fitness = sum([individual.fitness for individual in population])
  r = random.randint(0, int(population_fitness))
  prob = [individual.fitness/population_fitness for individual in population]
  
  for individual in population:
    s+=individual.fitness
    if s>=r:
      return individual

def TournamentSelection(population):
  K =  random.randint(1, len(population))
  k_individuals = []
  for _ in range(K):
    k_individuals.append(population[random.randint(0, len(population)-1)])
  k_individuals.sort(key = lambda x:x.fitness)

  return k_individuals[-1]

def Crossover(parent1,parent2,k=2):
  points = [np.random.randint(0,len(parent1.chromosome)) for _ in range(k)]
  points.sort()
  
  for i in points:
    A=np.append(parent1.chromosome[:i],parent2.chromosome[i:])
    B=np.append(parent2.chromosome[:i],parent1.chromosome[i:])
  return Individual(A),Individual(B)

def Repetition(old,new,rpgenome):

  if old:
    if np.array_equal(old.chromosome,new.chromosome):
      rpgenome +=1
    
    if new.fitness > old.fitness:
      old = new
      rpgenome=1
  else:
    old = new
  return rpgenome,old

def Main():
  POPULATION_SIZE = 100
  generations = 0
  end = False
  population = initialize_population(POPULATION_SIZE)
  best_individual = None
  new_best = None
  
  repeated_genome = 0
  while not end:

    if generations==1000:
      end = True

    
    
    if repeated_genome >=50:
      end=True
      print("Interrupt after repitition")

    fitnessFunc(population,tf__idf)

    new_generation = []
    population = sorted(population, key = lambda x:x.fitness)
    
    new_best = population[-1]

    repeated_genome,best_individual = Repetition(best_individual,new_best,repeated_genome)

    s = int((10*POPULATION_SIZE)/100)
    new_generation.extend(population[:s])
    
    for _ in range((POPULATION_SIZE-s)//2):
      parent1 = TournamentSelection(population)
      parent2 = TournamentSelection(population)
      offspring1,offspring2 = Crossover(parent1,parent2)
      new_generation.append(offspring1)
      new_generation.append(offspring2)
    
    print("Generation:",generations,"Best_individual:",best_individual.fitness)
    population = new_generation
    generations+=1



In [ ]:
 
if __name__ == "__main__":
  Main() 